# Exercise 1
Read and parse the chemical reactions `.xml` input file `rxns.xml`.


1. Collect the species into a species list.  My output is `['H', 'O', 'OH', 'H2', 'O2']`.
    
    Some notes and hints:
    * **Hint:**  For this `.xml` format you should have a loop over the `phase` element.
    * **Hint:** You can use the `find()` method to get the species array.

2. Calculate and print out the Arrhenius reaction rate coefficients using $R = 8.314$ and $T = 1500$.

    Some notes and hints:
    * **Hint:**  For this `.xml` format you should have loops over the `reactionData` element, the `reaction` element, the `rateCoeff` element, and the `Arrhenius` element using the `findall()` method discussed in lecture.
    * **Hint:** You can use the `find()` method to get the reaction rate coefficients.
    * My solution is:
    
        `k for reaction01 =    6.8678391864294477e+05
    k for reaction02 =    2.3105559199959813e+06`

In [1]:
import xml.etree.ElementTree as ET
import math

tree = ET.parse('rxns.xml')

phase = tree.find('phase')
species_arr = tree.find('speciesArray')

for child in phase:
    species_list = child.text.strip().split(' ')

species_list

['H', 'O', 'OH', 'H2', 'O2']

In [2]:
def get_reaction_rate_coeff(T, params, R=8.314): 
    
    if T <= 0: 
        raise ValueError('Temperature T must be positive.')
    if len(params) != 3:
        raise Exception('To get modified Arrhenius reaction rate, the length of params must be 3 in the order of [A, b, E].')
    if params[0] <= 0:
        raise ValueError('The Arrhenius prefactor A must be positive.')
    if isinstance(params[1], complex) and params[1].imag != 0:
        raise ValueError('The modified Arrhenius parameter b must be real.')
    if R != 8.314:
        warnings.warn('Warning: R = 8.314, the ideal gas constant should never be changed except to convert units.')
    A = params[0]
    b = params[1]
    E = params[2]
    
    try:
        ans = A * pow(T, b) * math.exp((-1)*E/(R*T))
        if ans == 0: 
            warnings.warn('Underflow: k is too close to 0, return value is set to 0.')
    except OverflowError:
        warnings.warn('Overflow: k is too large, return value is set to float(\'inf\').')
        ans = float('inf')
    return ans

In [3]:
from decimal import Decimal

T = 1500
reaction_data = tree.find('reactionData')
reactions = reaction_data.findall('reaction')
for reaction in reactions:
    rateCoeffs = reaction.findall('rateCoeff')
    for rateCoeff in rateCoeffs:
        arrheniuses = rateCoeff.findall('Arrhenius')
        for arrhenius in arrheniuses:
            A = float(arrhenius.find('A').text)
            b = float(arrhenius.find('b').text)
            E = float(arrhenius.find('E').text)
            params = [A, b, E]
            print('k for {} = {:.16e}'.format(reaction.get('id'), Decimal(get_reaction_rate_coeff(T, params))))



k for reaction01 = 6.8678391864294477e+5
k for reaction02 = 2.3105559199959813e+6
